In [115]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, robust_scale
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score,accuracy_score,auc,make_scorer,log_loss


In [41]:
np.zeros((1,30))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]])

In [60]:
dir_name = r"C:\Users\basil\esus\Data\Tennis\dummy\atp_matches_2016.csv"

In [61]:
data = pd.read_csv(dir_name)
winner_column = [col for col in data.columns if 'winner_' in col]
loser_column = [col for col in data.columns if 'loser_' in col]
column_not_available = ['score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced']
def get_column_name_winner(column_name, winner_indicator = 'winner_', loser_indicator = 'loser_'):
    if winner_indicator in column_name:
        return 'player1_'+column_name[len(winner_indicator):]
    elif loser_indicator in column_name:
        return 'player2_'+column_name[len(loser_indicator):]
    else:
        return column_name

def get_column_name_loser(column_name, winner_indicator = 'winner_', loser_indicator = 'loser_'):
    if winner_indicator in column_name:
        return 'player2_'+column_name[len(winner_indicator):]
    elif loser_indicator in column_name:
        return 'player1_'+column_name[len(loser_indicator):]
    else:
        return column_name
    
def get_training_data_from_pandas(df, winner_column, loser_column, column_not_available):
    column_to_keep = [col for col in df.columns if col not in column_not_available]
    df = df[column_to_keep]
    training_df_columns = [get_column_name_winner(col) for col in column_to_keep] + ['Player1_won'] 
    training_df = pd.DataFrame(data=None,columns=training_df_columns)
    df_for_row_winner = pd.DataFrame(data=np.zeros((1,len(training_df_columns))),columns=training_df_columns)
    df_for_row_loser = pd.DataFrame(data=np.zeros((1,len(training_df_columns))),columns=training_df_columns)
    for i in range(len(df)):
        for col in df.columns:
            df_for_row_winner[get_column_name_winner(col)] = df[col].loc[i]
            df_for_row_loser[get_column_name_loser(col)] = df[col].loc[i]
        df_for_row_loser['Player1_won'] = 0
        df_for_row_winner['Player1_won'] = 1
        training_df = training_df.append(df_for_row_loser)
        training_df = training_df.append(df_for_row_winner)
    return training_df

In [62]:
training_df = get_training_data_from_pandas(data, winner_column, loser_column, column_not_available)

In [64]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5882 entries, 0 to 0
Data columns (total 28 columns):
tourney_id             5882 non-null object
tourney_name           5882 non-null object
surface                5882 non-null object
draw_size              5882 non-null object
tourney_level          5882 non-null object
tourney_date           5882 non-null object
match_num              5882 non-null object
player1_id             5882 non-null object
player1_seed           2073 non-null float64
player1_entry          827 non-null object
player1_name           5882 non-null object
player1_hand           5868 non-null object
player1_ht             4180 non-null float64
player1_ioc            5882 non-null object
player1_age            5870 non-null float64
player1_rank           5820 non-null float64
player1_rank_points    5820 non-null float64
player2_id             5882 non-null object
player2_seed           2073 non-null float64
player2_entry          827 non-null object
player2_name

In [77]:
drop_second_path = ['player2_entry','player1_entry','player2_seed','player1_seed','tourney_name','tourney_id','surface','draw_size','tourney_date','match_num','tourney_level','player1_ioc','player2_ioc']
training_df_with_correct_data = training_df[[col for col in training_df.columns if col not in drop_second_path]]

In [78]:
training_df_with_correct_data = training_df_with_correct_data.dropna()
training_df_with_correct_data

player1_id           player1_name player1_hand  player1_ht  player1_age  \
0      103819          Roger Federer            R       185.0    34.406571   
0      105683           Milos Raonic            R       196.0    25.021218   
0      106071          Bernard Tomic            R       193.0    23.203285   
0      105683           Milos Raonic            R       196.0    25.021218   
0      105777        Grigor Dimitrov            R       188.0    24.637919   
0      103819          Roger Federer            R       185.0    34.406571   
0      105453          Kei Nishikori            R       178.0    26.015058   
0      106071          Bernard Tomic            R       193.0    23.203285   
0      104735           Tobias Kamke            R       180.0    29.623546   
0      103819          Roger Federer            R       185.0    34.406571   
0      104678         Viktor Troicki            R       193.0    29.897331   
0      105777        Grigor Dimitrov            R       188.0    24.637919   
0      104471             Ivan Dodig            R       183.0    31.003422   
0      105683           Milos Raonic            R       196.0    25.021218   
0      103285         Radek Stepanek            R       185.0    37.103354   
0      106071          Bernard Tomic            R       193.0    23.203285   
0      105062      Mikhail Kukushkin            R       183.0    28.024641   
0      105453          Kei Nishikori            R       178.0    26.015058   
0      105449          Steve Johnson            R       188.0    26.028747   
0      104678         Viktor Troicki            R       193.0    29.897331   
0      104468           Gilles Simon            R       183.0    31.019849   
0      105777        Grigor Dimitrov            R       188.0    24.637919   
0      105064        Thomaz Bellucci            L       188.0    28.013689   
0      105676           David Goffin            R       163.0    25.075975   
0      103917          Nicolas Mahut            R       190.0    33.952088   
0      106071          Bernard Tomic            R       193.0    23.203285   
0      105583          Dusan Lajovic            R       180.0    25.514031   
0      103285         Radek Stepanek            R       185.0    37.103354   
0      104797          Denis Istomin            R       188.0    29.325120   
0      105062      Mikhail Kukushkin            R       183.0    28.024641   
..        ...                    ...          ...         ...          ...   
0      104586            Lukas Rosol            R       196.0    30.600958   
0      104259  Philipp Kohlschreiber            R       178.0    32.372348   
0      104607          Tomas Berdych            R       196.0    30.450376   
0      104259  Philipp Kohlschreiber            R       178.0    32.372348   
0      104312          Andreas Seppi            R       190.0    32.386037   
0      105643      Federico Delbonis            L       190.0    25.765914   
0      104338            Juan Monaco            R       185.0    32.284736   
0      104926          Fabio Fognini            R       178.0    29.133470   
0      104926          Fabio Fognini            R       178.0    29.133470   
0      105643      Federico Delbonis            L       190.0    25.765914   
0      104678         Viktor Troicki            R       193.0    30.050650   
0      105062      Mikhail Kukushkin            R       183.0    28.177960   
0      105062      Mikhail Kukushkin            R       183.0    28.177960   
0      104925         Novak Djokovic            R       188.0    28.774812   
0      103843      Marco Chiudinelli            R       185.0    34.469541   
0      103893          Paolo Lorenzi            R       183.0    34.206708   
0      106058              Jack Sock            R       185.0    23.430527   
0      106071          Bernard Tomic            R       193.0    23.356605   
0      106071          Bernard Tomic            R       193.0    23.356605   
0      104545            

In [80]:
dummy_data = pd.get_dummies(training_df_with_correct_data[['player1_id','player2_id','player2_hand','player1_hand']])

In [82]:
scale_data = robust_scale(training_df_with_correct_data[['player1_ht','player1_age','player1_rank','player1_rank_points','player2_ht','player2_age','player2_rank','player2_rank_points']])

In [104]:
X = np.append(dummy_data.values,scale_data,axis=1)
Y = training_df_with_correct_data['Player1_won'].astype(int).values
X.shape


(3044, 314)

In [178]:
logist = LogisticRegression(penalty='l1',C = 1e-1)
logist.fit(X,Y)
Y_pred = [0.5 for i in range(len(Y))]
Y_pred = [0.1 for i in range(len(Y))]
Y_pred_2 = logist.predict(X)
for_score = make_scorer(accuracy_score)
cross_val_score(logist,X,y=Y,scoring=for_score)

array([ 0.68897638,  0.66272189,  0.66272189])

In [179]:
accuracy_score(Y,Y_pred_2)

0.68265440210249673

In [180]:
logist.coef_

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 